In [247]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import string

In [249]:
df = pd.DataFrame()
site = 'https://www.hockey-reference.com'
prev_player = ''
for letter in string.ascii_lowercase:
    count = 0
    url = requests.get(site+'/players/'+letter+'/').text
    soup = BeautifulSoup(url,'lxml')
    player_list = soup.find_all('p', class_='nhl')

    for count, player in enumerate(player_list):
        # Remove goalies and players from before 1980
        if int(player.text.split('(')[-1][0:4])<1980 or player.text[-2] == 'G':
            continue
        # If players have the same name, differentiate them
        if player.a.text == prev_player:
            if same_name_flag == True:
                same_name_count += 1
            same_name_flag = True
        else:
            same_name_flag = False
            same_name_count = 0
        player_name = player.a.text+same_name_flag*str(same_name_count)
        prev_player = player.a.text
        
        print(f'\r{count:03d}/{len(player_list)} of the letter {letter.upper()} completed, currently on: {player_name}                           ',end='')
        
        # Access player webpage
        player_url = site+player.a['href']
        player_url = requests.get(player_url).text
        player_page = BeautifulSoup(player_url,'lxml')   
        player_year_log = player_page.find('li',class_ = 'full hasmore')
        
        # Loop over all years in career
        for year in player_year_log.find_all('li'):
            if not year.text == 'Career Playoffs':
                game_log_url = site + year.a['href']
                game_log_url = requests.get(game_log_url).text
                # Get table of games for given year
                try:
                    df_list = pd.read_html(game_log_url, skiprows = 1, header = 0)
                except:
                    continue
                # Clean results
                tmp_df = df_list[0]
                tmp_df.rename(columns = {'Unnamed: 5':'Location','Unnamed: 7':'Result', 'G.1':'Goals','A':'Assists'}, inplace = True)
                tmp_df.drop(columns = ['+/-', 'EV', 'PP', 'SH', 'GW', 'EV.1', 'PP.1','SH.1', 'S', 'S%', 'SHFT', 'TOI',
                                   'HIT', 'BLK', 'FOW', 'FOL', 'FO%'], inplace=True)
                tmp_df.drop(tmp_df.loc[tmp_df['Rk']=='Rk'].index, inplace=True)
                tmp_df['Location'] = tmp_df.Location.apply(lambda x: 'A' if x == '@' else 'H')
                tmp_df['Age'] = tmp_df.Age.apply(lambda x: x[0:2])
                tmp_df['Player'] = player_name
                tmp_df[['G', 'Age', 'Goals', 'Assists', 'PTS', 'PIM']] = tmp_df[['G', 'Age', 'Goals', 'Assists', 'PTS', 'PIM']].apply(pd.to_numeric)
                
                # Append to df
                # TODO: Check if it would be better to just directly append to csv
                if df.empty:
                    df = tmp_df
                else:
                    df = df.append(tmp_df, ignore_index=True)
                    
print(f'\rCompleted                                                                                   ',end='')

059/60 of the letter Z completed, currently on: Andrei Zyuzin                                            

In [250]:
df.to_csv('NHL_game_log_all.csv',index=False)